https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2025/10-kubernetes/homework.md

In this homework, we'll deploy the lead scoring model from the homework 5.

## Building the image

In [1]:
!docker build -f Dockerfile_full -t zoomcamp-model:3.13.10-hw10 .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile_full                  0.0s
 => => transferring dockerfile: 411B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.13.10-slim-bo  0.1s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.1s
[+] Building 0.3s (1/4)                                          docker:default
 => [internal] load build definition from Dockerfile_full                  0.0s
 => => transferring dockerfile: 411B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.13.10-slim-bo  0.2s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.2s
 => [auth] astral-sh/uv:pull token for ghcr.io                             0.0s
[+] Building 0.5s (3/5)                 

## Question 1

Run it to test that it's working locally:

docker run -it --rm -p 9696:9696 zoomcamp-model:3.13.10-hw10

In [3]:
!python q6_test.py

{'conversion_probability': 0.49999999999842815, 'conversion': False}


Ans: the probability of getting a subscription is 0.49

## Installing `kubectl` and `kind`

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

In [5]:
!kubectl version --client

Client Version: v1.34.3
Kustomize Version: v5.7.1


What's the version of kind that you have?

In [6]:
!kind --version

kind version 0.30.0


Ans: 0.30.0

## Creating a cluster

Now let's create a cluster with `kind`:

In [7]:
!kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.34.0) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a question, bug, or feature request? Let us know! https://kind.sigs.k8s.io/#community 🙂


And check with `kubectl` that it was successfully created:

In [8]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:43665
CoreDNS is running at https://127.0.0.1:43665/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3

What's the smallest deployable computing unit that we can create and manage in Kubernetes (`kind` in our case)?

Ans: In Kubernetes, the smallest deployable and manageable unit is a __Pod__.

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

In [9]:
!kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   21m


Ans: ClusterIP

## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:3.13.10-hw10`), we need to register it with `kind`.

What's the command we need to run for that?

In [11]:
!kind load docker-image zoomcamp-model:3.13.10-hw10

Image: "zoomcamp-model:3.13.10-hw10" with ID "sha256:81a8079a43fc36e014193b389ea7afa39c710ddc038b1af9d261e54d5703b91d" not yet present on node "kind-control-plane", loading...


Ans: kind load docker-image

## Question 6

Now let's create a deployment config: [deployment.yaml](./deployment.yaml)

What is the value for `Port`?

Ans: 9696

## Question 7

Let's create a service for this deployment: [service.yaml](./service.yaml)

Answer: subscription

## Testing the service

We can test our service locally by forwarding the port 9696 on our computer to the port 80 on the service:

In [15]:
!kubectl port-forward service/kubernetes 9696:80

error: cannot attach to *v1.Service: invalid service 'kubernetes': Service is defined without a selector
